In [1]:
// simple log function
const log: <T>(s: T, prefix?: string) => void = (s, prefix?) =>
  console.log(
    (prefix ? prefix : "") + ": " +
      JSON.stringify(s, null, 2),
  );

//function logDocImpl(doc: DocImpl) {
//  console.log("docImpl: entityId=" + doc.entityId + ", value=" + doc.value + ", space=" + doc.space + ", sourceCell=" + doc.sourceCell + ", ephemeral=" + doc.ephemeral);
//}
function logDocImpl(doc: DocImpl, prefix?: string) {
  let logString = "docImpl: \n" +
    "  entityId=" + JSON.stringify(doc.entityId) + "\n" +
    "  space=" + doc.space + "\n" +
    "  sourceCell=" + doc.sourceCell + "\n" +
    "  ephemeral=" + doc.ephemeral;

  // Check if value is an array and add each object individually to the log string
  if (Array.isArray(doc.value)) {
    logString += "\n    value contains " + doc.value.length + " objects:";
    doc.value.forEach((item, index) => {
      // Convert to JSON string with 2-space indentation
      const jsonString = JSON.stringify(item, null, 2);

      // Split the JSON string into lines
      const jsonLines = jsonString.split("\n");

      // Add the first line with the value[index] prefix
      logString += `\n    value[${index}]: ${jsonLines[0]}`;

      // For remaining lines, add 4 additional spaces to maintain consistent indentation
      for (let i = 1; i < jsonLines.length; i++) {
        logString += "\n      " + jsonLines[i];
      }
    });
  } else {
    // For non-array values, apply the same indentation logic
    const jsonString = JSON.stringify(doc.value, null, 2);
    const jsonLines = jsonString.split("\n");

    // Add the first line with the value: prefix
    logString += "\n  value: " + jsonLines[0];

    // For remaining lines, add proper indentation
    for (let i = 1; i < jsonLines.length; i++) {
      logString += "\n    " + jsonLines[i];
    }
  }
  console.log((prefix ? prefix : "") + ": " + logString);
}

### Introduction

This exploration tries to show what the various "parts" of the system does. It
tries to make explicit all the things that usually happen in the backgroun.

This means we won't be using the highest level abstractions. I believe this will
show what's really happening under the hood and give you a better idea of what's
in the guts of the system.

### List of Charms

The first thing I'd like to do is be able to see a list of our charms, just like
when you go to the toolshed common knowledge page.

The call I'd like to do is `getDoc<CellLink[]>([], "charms", SPACE);`

BUT before we can access charms, we need to set up our connection to the
database. Let's do that. NOTE: notice that `signer` is undefined in the Storage
object. We'll get back to that later.

In [ ]:
import { storage } from "../runner/src/storage.ts";

// where our server is
const API_URL = "https://toolshed.saga-castor.ts.net/";

// `storage` is a singleton and you have to give it the server URL
storage.setRemoteStorage(new URL(API_URL));
storage; // notice signer is undefined

StorageImpl {
  storageProviders: Map(0) {},
  remoteStorageUrl: URL {
    href: "https://toolshed.saga-castor.ts.net/",
    origin: "https://toolshed.saga-castor.ts.net",
    protocol: "https:",
    username: "",
    password: "",
    host: "toolshed.saga-castor.ts.net",
    hostname: "toolshed.saga-castor.ts.net",
    port: "",
    pathname: "/",
    hash: "",
    search: ""
  },
  signer: undefined,
  docIsSyncing: Set(0) {},
  docIsLoading: Map(0) {},
  loadingPromises: Map(0) {},
  loadingResolves: Map(0) {},
  writeDependentDocs: Map(0) {},
  writeValues: Map(0) {},
  readDependentDocs: Map(0) {},
  readValues: Map(0) {},
  currentBatch: [],
  currentBatchProcessing: false,
  currentBatchResolve: [Function (anonymous)],
  currentBatchPromise: Promise { <pending> },
  lastBatchTime: 0,
  lastBatchDebounceCount: 0,
  debounceTimeout: null,
  batchStartTime: 0,
  cancel: [Function: cancelAll],
  addCancel: [Function: addCancel]
}

### Space

Spaces are in flux and we'll have private and public spaces, but for now, we
still have the ability to pass in a simple string for the space. We'll use
"common-knowledge" which is our default space.

### getDoc()

Now we have storage set up "enough" to call getDoc(value, cause, space). The
value we pass in here is basically the empty value `[]` which is the same as
undefined. The cause is a special hardcoded string "charms" that we use.
CharmManager has this hardcoded string in it. Lastly we pass in the space which
we already talked about.

In [3]:
import { DocImpl, getDoc } from "../runner/src/doc.ts";
import { createRef } from "../runner/src/doc-map.ts";

// space is usually a did now, but it still accepts a string
const SPACE = "common-knowledge";

const allCharmsDoc: DocImpl<CellLink[]> = getDoc<CellLink[]>(
  [],
  "charms",
  SPACE,
);
allCharmsDoc;

{
  get: [Function: get],
  getAsQueryResult: [Function: getAsQueryResult],
  asCell: [Function: asCell],
  send: [Function: send],
  updates: [Function: updates],
  getAtPath: [Function: getAtPath],
  setAtPath: [Function: setAtPath],
  freeze: [Function: freeze],
  isFrozen: [Function: isFrozen],
  toJSON: [Function: toJSON],
  value: [Getter],
  entityId: [Getter/Setter],
  space: [Getter/Setter],
  sourceCell: [Getter/Setter],
  ephemeral: [Getter/Setter],
  copyTrap: [Getter],
  registerSchemaUse: [Function: registerSchemaUse],
  [Symbol(toOpaqueRef)]: [Function: [toOpaqueRef]],
  [Symbol(isDoc)]: true
}

### More on getDoc()

How does getDoc() find the doc it's looking for? As we discussed, getDoc()
received a `value` and `cause` parameters.

Within getDoc(), we take these two parameters and create an `EntityId` from
them. What this means is that if we call getDoc() with different value and cause
parameters, we should get different `EntityId`s. Let's test this out. We'll get
another "well known" document for the pinned charms. If we're right, the IDs
will be different.

In [4]:
import { deepEqual } from "../builder/src/utils.ts";

// get the pinned charms doc
const pinsDoc: DocImpl<CellLink[]> = getDoc<CellLink[]>(
  [],
  "pinned-charms",
  SPACE,
);

// see that the entityId for both allCharmsDoc and pinsDoc are different
log(allCharmsDoc.entityId, "allCharmsDoc entity id");
log(pinsDoc.entityId, "pinsDoc entity id");
log(deepEqual(allCharmsDoc, pinsDoc), "is the same?");

allCharmsDoc entity id: {
  "/": "baedreiahv63wxwgaem4hzjkizl4qncfgvca7pj5cvdon7cukumfon3ioye"
}
pinsDoc entity id: {
  "/": "baedreihxpwcmhvzpf5weuf4ceow4zbahqikvu5ploox36ipeuvqnminyba"
}
is the same?: false


### createRef

Let's continue our detour, how do EntityIds get created. This uses the enigmatic
merkle-reference. We don't need to understand how merkle-references are made,
just that it creates a unique digital fingerprint of the Doc, like a hash.

Let's recreate the allCharmsDocEntityId from the "value" and "cause" arugments
to getDoc. As a reminder, this was our getDoc call:

```ts
const allCharmsDoc: DocImpl<CellLink[]> = getDoc<CellLink[]>(
  [],
  "charms",
  SPACE,
);
```

So we should be able to pass in [] and "charms" to createRef() and get back the
same entity ID. Let's try it.

In [5]:
const recreatedEntityId: EntityId = createRef([], "charms");
log(recreatedEntityId, "recreated allCharmsDoc entityid");
log(allCharmsDoc.entityId, "original allCharmsDoc entityid");
log(deepEqual(recreatedEntityId, allCharmsDoc.entityId), "are they the same?");

recreated allCharmsDoc entityid: {
  "/": "baedreiahv63wxwgaem4hzjkizl4qncfgvca7pj5cvdon7cukumfon3ioye"
}
original allCharmsDoc entityid: {
  "/": "baedreiahv63wxwgaem4hzjkizl4qncfgvca7pj5cvdon7cukumfon3ioye"
}
are they the same?: true


### Spaces dont matter

Did you notice that we don't care about spaces when we make the id? If I use a
different space for the original allCharmsDoc, I should get back the same
EntityID from getDoc(). Let's verify!

In [6]:
const differentSpaceDoc: DocImpl<CellLink[]> = getDoc<CellLink[]>(
  [],
  "charms",
  "some other space",
);
log(
  differentSpaceDoc.entityId,
  "entityid for allCharmsDoc but with different space",
);
log(
  allCharmsDoc.entityId,
  "entityid for original allCharmsDoc in the 'charms' space",
);
log(
  deepEqual(differentSpaceDoc.entityId, allCharmsDoc.entityId),
  "are they the same?",
);

entityid for allCharmsDoc but with different space: {
  "/": "baedreiahv63wxwgaem4hzjkizl4qncfgvca7pj5cvdon7cukumfon3ioye"
}
entityid for original allCharmsDoc in the 'charms' space: {
  "/": "baedreiahv63wxwgaem4hzjkizl4qncfgvca7pj5cvdon7cukumfon3ioye"
}
are they the same?: true


To further demystify getDoc(), the parameters we pass in really can be anything.

In [7]:
const some_val = ["arrays", "are", { value: "good" }];
const some_cause = { some: "random", value: 42 };
const some_space = "random string in here";
const crazyDoc = getDoc<number>(some_val, some_cause, some_space);
log(crazyDoc, "crazydoc");

// can try changing val, cause, space and see that nested objects change merke-reference

crazydoc: {
  "/": "baedreiekup34zo6rx6l33orhkktrzxgnrxxt7p3rmlnqylb4dv2swcc4kq"
}


In [8]:
// let's see what this crazy doc looks like
logDocImpl(crazyDoc);

: docImpl: 
  entityId={"/":"baedreiekup34zo6rx6l33orhkktrzxgnrxxt7p3rmlnqylb4dv2swcc4kq"}
  space=random string in here
  sourceCell=undefined
  ephemeral=false
    value contains 3 objects:
    value[0]: "arrays"
    value[1]: "are"
    value[2]: {
        "value": "good"
      }


In [9]:
// we'll set this charmsDoc to the doc we want to use from here on to get the list of charms
// either to pinsDoc or allCharmsDoc
const charmsDoc = pinsDoc;

### Back on track

Back from the short detour, we want to continue to get the charms list. Right
now, we have `charmsDoc` that has an EntityID but we haven't loaded the doc from
Storage yet (the database). So let's do that now. However, we are...

### Stuck on Storage

We're making a point to access data via the `Storage` interface rather than
through CharmManager or other possibly more convenient methods. Thus, our next
step is to call `Storage.syncCell(docImpl);` This would let us get the data for
the doc. However, this would crash with an exception. Jake ran into this the
hard way and the system somehow ate the exception.

### Another short detour, questioning our Identity

Remember when we saw the signer property was undefined in storage before? We
have to set that now in order to call syncCell. What we need to call `setSigner`
on `storage`. We create a signer via the Identity module.

In [ ]:
import { DIDKey, Identity } from "../identity/src/index.ts";

// key system similar to mneumonic registration
const user_identity: Identity<DIDKey> = await Identity.fromPassphrase(
  "some passphrase",
);

// NOTE: we can use this key directly with storage, however, to better replicate
// the long term goals, we derive a new key from the root key
// this is a lot like how users will have their own keys for each of their spaces
// we would normally pass in the space name (which is like the "common-knowledge" part of the URL)
const space_key = await user_identity.derive(SPACE);

// helper function? why not
function initializeStorage(
  storage: Storage,
  remoteStorageURL: string,
  signer: Signer,
): Storage {
  storage.setRemoteStorage(new URL(remoteStorageURL));
  storage.setSigner(signer);
  return storage;
}

// use `space_key` the derived space-based identity as the signer for the storage
const myStorage = initializeStorage(storage, API_URL, space_key);

log(user_identity, "user identity (looks empty but its just private fields");
myStorage; // notice signer is no longer undefined

user identity (looks empty but its just private fields: {}


StorageImpl {
  storageProviders: Map(0) {},
  remoteStorageUrl: URL {
    href: "https://toolshed.saga-castor.ts.net/",
    origin: "https://toolshed.saga-castor.ts.net",
    protocol: "https:",
    username: "",
    password: "",
    host: "toolshed.saga-castor.ts.net",
    hostname: "toolshed.saga-castor.ts.net",
    port: "",
    pathname: "/",
    hash: "",
    search: ""
  },
  signer: Identity {},
  docIsSyncing: Set(0) {},
  docIsLoading: Map(0) {},
  loadingPromises: Map(0) {},
  loadingResolves: Map(0) {},
  writeDependentDocs: Map(0) {},
  writeValues: Map(0) {},
  readDependentDocs: Map(0) {},
  readValues: Map(0) {},
  currentBatch: [],
  currentBatchProcessing: false,
  currentBatchResolve: [Function (anonymous)],
  currentBatchPromise: Promise { <pending> },
  lastBatchTime: 0,
  lastBatchDebounceCount: 0,
  debounceTimeout: null,
  batchStartTime: 0,
  cancel: [Function: cancelAll],
  addCancel: [Function: addCancel]
}

### Lessgo Sync!

Now we are all clear to call syncCell! syncCell returns a Promise(), so we await
on it before accessing the object.

While it's called syncCell, it really syncs either DocImpls or Cells. We'll get
into that more later.

```ts
interface Storage {
    syncCell<T>(
        subject: DocImpl<T> | Cell<any>,
        expectedInStorage: boolean = false,
      ): Promise<DocImpl<T>> | DocImpl<T> 
}
```

In [11]:
// start syncing on this document
// notice that we call syncCell on a DocImpl
// also, we have to await on this before accessing otherwise we may get a race condition!
logDocImpl(charmsDoc, "before syncCell");
await myStorage.syncCell(charmsDoc);
logDocImpl(charmsDoc, "after syncCell");

before syncCell: docImpl: 
  entityId={"/":"baedreihxpwcmhvzpf5weuf4ceow4zbahqikvu5ploox36ipeuvqnminyba"}
  space=common-knowledge
  sourceCell=undefined
  ephemeral=false
    value contains 0 objects:
after syncCell: docImpl: 
  entityId={"/":"baedreihxpwcmhvzpf5weuf4ceow4zbahqikvu5ploox36ipeuvqnminyba"}
  space=common-knowledge
  sourceCell=undefined
  ephemeral=false
    value contains 4 objects:
    value[0]: {
        "cell": {
          "/": "baedreietbxsdgmw67da47dp6aqervnk2zyppevhx4p4mn3pet6onvaxcmy"
        },
        "path": []
      }
    value[1]: {
        "cell": {
          "/": "baedreih6pdtzk4xvjuxlsp3htrugtipgndn6xtd4cwbdr6wgscb6fh2nlu"
        },
        "path": []
      }
    value[2]: {
        "cell": {
          "/": "baedreiel5fi755jovxnou5bpdsuvcylibayiaaipaysi2tiip26ik3b6ya"
        },
        "path": []
      }
    value[3]: {
        "cell": {
          "/": "baedreid2qh2k6gcie4ezckqukkplpeqlz7jnxirt7aehxdcw4sfg5aszh4"
        },
        "path": []
      }


### Document Loaded but not the Charms

We have retrieved the document now, however, it doesn't actually contain the
charms, just references to them as you can see via the `value` fields. Each
element in the value array

We have to look at each of the elements in the `value` array and load each one.
Each one has the format:

```ts
{
    "cell": {
      "/": "baedreid2qh2k6gcie4ezckqukkplpeqlz7jnxirt7aehxdcw4sfg5aszh4"
    },
    "path": []
}
```

This is a CellLink, which you can find in runner/src/cell.ts:

```ts
/**
 * Cell link.
 *
 * A cell link is a doc and a path within that doc.
 */
export type CellLink = {
  space?: string;
  cell: DocImpl<any>;
  path: PropertyKey[];
};
```

We sometimes call these DocLinks, they are the same thing. In cell.ts, you'll
still see

```
* @method getAsDocLink Returns a document link for the cell.
* @returns {CellLink}
```

We'll take each of the CellLinks and call getDocByEntityId(), this will load the
docImpl for us. This time we do _NOT_ need to sync on each child because the
first syncCell also sync'd on all the child cells. NOTE: this logic of loading
dependencies happens in `runner/src/storage.ts:_processCurrentBatch()`

In [12]:
import { getDocByEntityId } from "../runner/src/doc-map.ts";

// lets load and look at each charm in the charms arrays
const allCharmsArray: CellLink[] = charmsDoc.value;
allCharmsArray.forEach(async (charmRef) => {
  // pull out the entity id from each element of the array
  const charmCellId: EntityId = charmRef["cell"];

  // create the empty docImpl that has the EntityId in it
  const charmCellDoc = getDocByEntityId(SPACE, charmCellId);
  logDocImpl(charmCellDoc);
});

: docImpl: 
  entityId={"/":"baedreietbxsdgmw67da47dp6aqervnk2zyppevhx4p4mn3pet6onvaxcmy"}
  space=common-knowledge
  sourceCell=[object Object]
  ephemeral=false
  value: {
      "$NAME": "Common Time",
      "$UI": {
        "type": "vnode",
        "name": "common-iframe",
        "props": {
          "src": "<html>\n<head>\n<script src=\"https://cdn.tailwindcss.com\"></script>\n<script crossorigin src=\"https://unpkg.com/react@18.3.1/umd/react.production.min.js\"></script>\n<script crossorigin src=\"https://unpkg.com/react-dom@18.3.1/umd/react-dom.production.min.js\"></script>\n<script src=\"https://unpkg.com/@babel/standalone/babel.min.js\"></script>\n<script>\nwindow.onerror = function (message, source, lineno, colno, error) {\n  window.parent.postMessage(\n    {\n      type: \"error\",\n      data: {\n        description: message,\n        source: source,\n        lineno: lineno,\n        colno: colno,\n        stacktrace: error && error.stack ? error.stack : new Error().stack,\

In [16]:
// lets see what the $UI.props.$context.$alias is about

//log(charm); //charm["$UI"]//.props["$context"]["$alias"]
manager.runPersistent(allCharmsArray[0].cell.value, undefined, "foo");

ReferenceError: manager is not defined